In [15]:
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC, SVR
import seaborn as sns
import os
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.impute import SimpleImputer
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from tqdm import tqdm
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, classification_report, roc_curve, \
RocCurveDisplay, roc_auc_score, r2_score, mean_absolute_error
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import log_loss
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.naive_bayes import BernoulliNB, GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.linear_model import ridge_regression, ElasticNet
from sklearn.linear_model import Ridge


In [16]:
df = pd.read_csv('../datasets/Boston.csv')
df.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [17]:
x,y = df.drop('medv', axis = 1), df['medv']
scaler = StandardScaler()
x = scaler.fit_transform(x)
x_train, x_test, y_train, y_test = train_test_split(x,y, random_state=25, test_size=0.3)

In [18]:
ridge = Ridge()
ridge.fit(x_train, y_train)
y_pred = ridge.predict(x_test)
print(mean_absolute_error(y_test, y_pred))

3.1007182235230206


In [19]:
elastic = ElasticNet()
elastic.fit(x_train, y_train)
y_pred = elastic.predict(x_test)
print(mean_absolute_error(y_test, y_pred))

3.4256480790282287


In [20]:
dtr1 = DecisionTreeRegressor(random_state=25)
dtr1.fit(x_train, y_train)
y_pred = dtr1.predict(x_test)
print(mean_absolute_error(y_test, y_pred))

2.9381578947368423


using voting classifier

In [21]:
voting = VotingRegressor([('Ridge', ridge),('EL', elastic),('DecisionTree', dtr1)])
voting.fit(x_train, y_train)
y_pred = voting.predict(x_test)
print(mean_absolute_error(y_test, y_pred))

2.613876274799628


applying wieghts

In [22]:
voting = VotingRegressor([('Ridge', ridge),('EL', elastic),('DecisionTree', dtr1)],
                         weights = [5,2,7])
voting.fit(x_train, y_train)
y_pred = voting.predict(x_test)
print(mean_absolute_error(y_test, y_pred))

2.5261238725243875


applying different weights

In [25]:
scores = []

for i in tqdm(range(1,11)):
    for j in range(1, 11):
        for k in range(1, 11):
            voting = VotingRegressor([('Ridge', ridge),('EL', elastic),('DecisionTree', dtr1)],
                                    weights = [i,j,k])
            voting.fit(x_train, y_train)
            y_pred = voting.predict(x_test)
            # print(mean_absolute_error(y_test, y_pred))
            scores.append([i,j,k,mean_absolute_error(y_test, y_pred)])

scores_df = pd.DataFrame(data = scores, columns = ['i','j','k','Accuracy Score'])
scores_df.sort_values('Accuracy Score', ascending=True)

100%|██████████| 10/10 [00:08<00:00,  1.23it/s]


,i,j,k,Accuracy Score
719,8,2,10,2.524121
304,4,1,5,2.524121
405,5,1,6,2.524210
506,6,1,7,2.524321
607,7,1,8,2.524405
...,...,...,...,...
180,2,9,1,3.082798
190,2,10,1,3.106290
70,1,8,1,3.107621
80,1,9,1,3.133713
